In [ ]:
%

In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

# 1. Sesión y rol
session = sagemaker.Session()
region = session.boto_region_name  # debería ser "us-east-1"
role = "arn:aws:iam::057176724057:role/Rol_ML_Cuantico_FullAccess"  # tu rol de SageMaker

# 2. Ruta al TAR del modelo en S3 (la que YA tienes)
s3_model_path = "s3://modelos-sagemaker/modelos/mnist/mnist_quantum/model_hybrid.tar.gz"

# 3. Definir el modelo en script mode
pytorch_model = PyTorchModel(
    model_data=s3_model_path,
    role=role,
    framework_version="1.13",           # coherente con el contenedor que usabas
    py_version="py39",
    entry_point="inference_quantum.py", # archivo dentro de source_dir
    source_dir="modelos/code/mnist/mnist_quantum/code",  # ruta LOCAL donde tienes inference_quantum.py y modelcnn.py
)

# 4. Desplegar el endpoint
predictor = pytorch_model.deploy(
    endpoint_name="mnist-hybrid-v2",    # nombre nuevo para no chocar con el viejo
    initial_instance_count=1,
    instance_type="ml.m5.large",        # o el que estabas usando
)

print("Endpoint creado:", predictor.endpoint_name)


TypeError: Metaclasses with custom tp_new are not supported.

In [2]:
import boto3
import botocore
import json
import base64
from botocore.config import Config

AWS_REGION = "us-east-1"
ENDPOINT_NAME = "mnist-hybrid"
TEST_IMAGE_PATH = "test_digit.png"  # pon aquí tu imagen de prueba

# Config con timeout para que no se quede “pegado” infinito
boto_config = Config(
    read_timeout=120,      # tiempo máximo esperando la respuesta
    retries={"max_attempts": 3}
)

runtime = boto3.client("sagemaker-runtime", region_name=AWS_REGION, config=boto_config)

# 1. Leer la imagen y convertirla a base64
with open(TEST_IMAGE_PATH, "rb") as f:
    image_bytes = f.read()

image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# 2. Payload EXACTO que espera tu input_fn: {"input": "<b64>"}
payload = {"input": image_b64}

print("Invocando endpoint:", ENDPOINT_NAME)

try:
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(payload),
    )

    status_code = response["ResponseMetadata"]["HTTPStatusCode"]
    body_str = response["Body"].read().decode("utf-8")

    print(f"\nHTTP status: {status_code}")
    print("Respuesta cruda:")
    print(body_str)

    try:
        body_json = json.loads(body_str)
        print("\nJSON parseado:")
        print(json.dumps(body_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("\n(No es JSON válido, se muestra como texto)")

except botocore.exceptions.ClientError as e:
    print("\n❌ Error al invocar el endpoint:")
    print("Código:", e.response["Error"]["Code"])
    print("Mensaje:", e.response["Error"]["Message"])
    print("RequestId:", e.response["ResponseMetadata"].get("RequestId"))
    raise
except botocore.exceptions.EndpointConnectionError as e:
    print("\n❌ Problema de conexión con SageMaker Runtime:")
    print(e)
    raise


Invocando endpoint: mnist-hybrid

❌ Error al invocar el endpoint:
Código: ModelError
Mensaje: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/mnist-hybrid in account 057176724057 for more information.
RequestId: 33443322-673e-43bd-b22e-7145fe127898


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/mnist-hybrid in account 057176724057 for more information.

# Navega al directorio que contiene la carpeta de tu modelo
# cd /ruta/a/la/carpeta/padre

# Crea el archivo .tar.gz usando el flag -h para dereferenciar los enlaces simbólicos
tar -h -czvf model.tar.gz my_model_dir/


In [3]:
import boto3

AWS_REGION = "us-east-1"
ENDPOINT_NAME = "mnist-hybrid"

sm = boto3.client("sagemaker", region_name=AWS_REGION)

resp = sm.describe_endpoint(EndpointName=ENDPOINT_NAME)
print("Estado:", resp["EndpointStatus"])
print("Último mensaje:", resp.get("FailureReason", "OK"))


ModuleNotFoundError: No module named 'boto3'

In [3]:
!pip install transformers torch pysentimiento
# Si estás usando una versión antigua de pip, usa %pip install en su lugar:
# %pip install transformers torch pysentimiento

  Using cached torch-2.9.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_

In [6]:
# Asegúrate de ejecutar esto en una celda antes de continuar
import sys
!{sys.executable} -m pip install transformers torch pysentimiento

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached pysentimiento-0.7.3-py3-none-any.whl.metadata (7.7 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.11.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached emoji-2.15.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached spacy-3.8.11-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (27 kB)
  Using cached httpx-0.28.1-py3-none-any.wh

In [3]:
## Script de Prueba Corregido (Con 'inference.py' como entry point)

import os
import json
import sys

# --- 1. CONFIGURACIÓN DE RUTAS ---
PROJECT_ROOT = "/home/ec2-user/SageMaker/Proyecto_Servidores" 

# Directorio que contiene el script inference.py
CODE_DIR = os.path.join(PROJECT_ROOT, 'modelos/sentimientos/model_pysentimiento/code')

# Directorio que contiene los artefactos del modelo (config.json, model.safetensors, etc.)
# ESTA RUTA DEBE CONTENER LOS ARCHIVOS DE HF. Si sigue fallando, es que NO ESTÁN allí.
HF_MODEL_DIR_ARTEFACTOS = os.path.join(PROJECT_ROOT, "modelos/sentimientos/model_pysentimiento")

TEST_TEXTS = ["Me siento genial hoy, es un día maravilloso.", 
              "Esta película fue terrible y aburrida."]


# --- 2. IMPORTACIÓN DE FUNCIONES ---
def get_inference_functions(code_dir):
    """Importa model_fn, predict_fn, y output_fn de forma dinámica."""
    if code_dir not in sys.path:
        sys.path.append(code_dir)
        
    try:
        # ¡IMPORTACIÓN CORREGIDA AL NUEVO NOMBRE DEL ARCHIVO!
        from inference import model_fn, predict_fn, output_fn
        print(f"✅ Importación exitosa de 'inference.py' desde: {code_dir}")
        return model_fn, predict_fn, output_fn
    except Exception as e:
        print(f"❌ ERROR en la importación. Asegúrate de que el archivo se llame 'inference.py' y que la instalación sea exitosa:\n{e}")
        return None, None, None

# Obtener las funciones
model_fn, predict_fn, output_fn = get_inference_functions(CODE_DIR)

# --- 3. EJECUCIÓN DE LA PRUEBA SIMULADA ---
if model_fn and predict_fn and output_fn:
    
    print("\n--- 1. SIMULACIÓN DE CARGA (model_fn) ---")
    print(f"Buscando archivos Hugging Face en: {HF_MODEL_DIR_ARTEFACTOS}")
    try:
        # LLAMADA CLAVE: Pasamos la ruta de los artefactos
        model_and_tokenizer = model_fn(HF_MODEL_DIR_ARTEFACTOS)
        
        print("✅ Modelo PyTorch/Hugging Face cargado exitosamente.")
        
        print("\n--- 2. SIMULACIÓN DE PREDICCIÓN (predict_fn) ---")
        predictions_raw = predict_fn(TEST_TEXTS, model_and_tokenizer)
        
        print("\n--- 3. SIMULACIÓN DE SALIDA (output_fn) ---")
        output_json, _ = output_fn(predictions_raw, "application/json")
        print(json.dumps(json.loads(output_json), indent=2))
        
    except Exception as e:
        print(f"❌ FALLO DE PRUEBA: Error durante la ejecución de model_fn o predict_fn.")
        print(f"Detalle del error: {e}")

✅ Importación exitosa de 'inference.py' desde: /home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento/code

--- 1. SIMULACIÓN DE CARGA (model_fn) ---
Buscando archivos Hugging Face en: /home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento
Buscando archivos Hugging Face en: /home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento
❌ FALLO DE PRUEBA: Error durante la ejecución de model_fn o predict_fn.
Detalle del error: Unrecognized model in /home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, bl

In [4]:
import os
MODEL_PATH = "/home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento"
print(f"Contenido del directorio de artefactos ({MODEL_PATH}):")

# Usamos el comando !ls -F para listar y mostrar si hay subcarpetas (con un / al final)
!ls -F {MODEL_PATH}

Contenido del directorio de artefactos (/home/ec2-user/SageMaker/Proyecto_Servidores/modelos/sentimientos/model_pysentimiento):
code/	      model.safetensors@	tokenizer.json@
config.json@  special_tokens_map.json@	tokenizer_config.json@


In [1]:
import boto3, json
client = boto3.client('sagemaker-runtime', region_name='us-east-1')
resp = client.invoke_endpoint(
    EndpointName='sentiment-hf-endpoint',
    ContentType='application/json',
    Body=json.dumps({"input": "es genial aprender a programar en python"})
)
print(resp['Body'].read())


ModuleNotFoundError: No module named 'boto3'